In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import readdata as rd

import datetime
time_origin = datetime.datetime(2010,1,1,0,0,0)
sys.path.append('/Users/ajc/Desktop/ATS/ats-repo/ats/tools/utils/')
import plot_water_table2D
sys.path.append(os.getcwd())
import SiteC_2012_2014WL 

In [3]:
reload(SiteC_2012_2014WL)

data2012 = SiteC_2012_2014WL.getData_2012()

In [4]:
reload(SiteC_2012_2014WL)
ObservedData2013 = dict()
data2013 = SiteC_2012_2014WL.getData_2013()

In [5]:
reload(SiteC_2012_2014WL)
ObservedData2014 = dict()
data2014 = SiteC_2012_2014WL.getData_2014()

In [150]:
ObservedData2012 = dict()
ObservedData2013 = dict()
ObservedData2014 = dict()
print data2013.keys()
for key in data2013.keys():
    ObservedData2012[key] = data2012[key]
    ObservedData2013[key] = data2013[key]
    ObservedData2014[key] = data2014[key]

['time_C40', 'time_C39', 'water_level_C39', 'water_level_C37', 'water_level_C40', 'time_C37']


In [151]:
simulation = 'SiteC_0R2Q'
reload(plot_water_table2D)
path = "/Users/ajc/Projects/ATS-Data/OR-CONDO/simulations/barrow_icewedges/NGEE/SiteC/"

Time, WaterLevelCenter, ThawDepthC, Z_c = plot_water_table2D.water_table2D(path + '%s/'%simulation)
Time, WaterLevelTrough, ThawDepthT, Z_t = plot_water_table2D.water_table2D(path + '%s/'%simulation, location='trough')

Location:  center 45
('Datum (surface elevation)', array([4.80721274]))
Location:  trough 1
('Datum (surface elevation)', array([4.71971499]))


In [152]:
cellid = 44
metdata = rd.GetSurfVarFromVis(path + '/%s/visdump_surface_data.h5'%simulation, 'surface-precipitation_rain', cellid=cellid)

In [153]:
# correcting the elevation of the center
for key in ObservedData2012.keys():
    if 'water_level_C40' in key:
        ObservedData2012[key + '_Elev'] = np.array([d - 4.91 + Z_c for d in ObservedData2012[key]])


In [154]:
DataSim =dict()
time = [3*365*0 + t*365.25 for t in Time]

DataSim['time'] = time
DataSim['WL_Center'] = WaterLevelCenter
DataSim['WL_Trough'] = WaterLevelTrough
DataSim['TD_Center'] = [t + Z_c for t in ThawDepthC]
        
D1_C = []
D1_T = []
T1_C = []
T1_T = []
for i,d in enumerate(DataSim['WL_Center']):
    if d != 0:
        D1_C.append(d + Z_c)
        T1_C.append(DataSim['time'][i])
for i,d in enumerate(DataSim['WL_Trough']):
    if d != 0:
        D1_T.append(d + Z_t)
        T1_T.append(DataSim['time'][i])

In [155]:
#--- Remove the assigned 100 to the missing data points
LC39 = len(ObservedData2012['water_level_C39'])
mask = np.ones(LC39, dtype=bool)

for i,d1 in enumerate(ObservedData2012['water_level_C39']):
    if d1 > 10:
        mask[i] = False

ObservedData2012['water_level_C39_mask'] = ObservedData2012['water_level_C39'][mask]
ObservedData2012['time_C39_mask'] = ObservedData2012['time_C39'][mask]

#-----------------------------------------------------------------
# Average the data at the two center wells
assert len(ObservedData2012['water_level_C39']) == len(ObservedData2012['water_level_C40_Elev'])
data = []
index = np.where(ObservedData2012['water_level_C39'] > 10)[0]

mid_value = 0.5*(ObservedData2012['water_level_C39'][index[0]-1] + ObservedData2012['water_level_C39'][index[-1]+1])

for d1,d2 in zip(ObservedData2012['water_level_C39'], ObservedData2012['water_level_C40_Elev']):
    sum = 0
    if d1 < 10:
        sum = 0.5*(d1+d2)
    else:
        sum = 0.5*(d2 + mid_value)
    data.append(sum)
ObservedData2012['water_level_Center_Elev'] = np.concatenate(data)

print len(ObservedData2013['water_level_C39']), len(ObservedData2013['water_level_C40'])
#assert len(ObservedData2013['water_level_C39']) == len(ObservedData2013['water_level_C40'])
data = []
c = 0
for i in range(len(ObservedData2013['water_level_C39'])):
    if i <7:
        data.append(ObservedData2013['water_level_C39'][i])
    elif (i < 113):
        sum = 0.5*(ObservedData2013['water_level_C39'][i] + ObservedData2013['water_level_C40'][i-7])    
        data.append(sum)
    else:
        data.append(ObservedData2013['water_level_C39'][i])


ObservedData2013['water_level_Center'] = np.array(data)

#--- 2014
print len(ObservedData2014['water_level_C39']), len(ObservedData2014['water_level_C40'])
data = []
for i in range(len(ObservedData2014['water_level_C39'])):
        sum = 0.5*(ObservedData2014['water_level_C39'][i] + ObservedData2014['water_level_C40'][i])    
        data.append(sum)

ObservedData2014['water_level_Center'] = np.array(data)

114 106
85 85


In [156]:

outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/'
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
axs = axs.ravel()
#fig, axs = plt.subplots(3,2,figsize=(10,8))


ax = plt.subplot(321)
plt.fill_between(ObservedData2012['time_C37'], ObservedData2012['water_level_C37']-0.03, 
                 ObservedData2012['water_level_C37']+ 0.03, color='c')
plt.plot(ObservedData2012['time_C37'], ObservedData2012['water_level_C37'],'r')
plt.plot(T1_T, D1_T,'k*',markersize='4')
plt.axhline(y=Z_t,linestyle='--',color='g')
plt.text(882, 4.6, 'Trough, 2012')
plt.ylabel('Water level')



#ax.set_xticks(np.arange(880, 1001, step=20))
#ax.set_xticklabels(timeadj2012)
ax.set_xlim(880, 1000)
#plt.margins(0.2)
plt.ylim(4.5, 5.1)


ax1 = plt.subplot(322)

plt.fill_between(ObservedData2012['time_C39'], ObservedData2012['water_level_Center_Elev']-0.03, 
                 ObservedData2012['water_level_Center_Elev']+ 0.03, color='c')
plt.plot(ObservedData2012['time_C39'], ObservedData2012['water_level_Center_Elev'],'r',label='Center, 2012')
#plt.plot(ObservedData2012['time_C40'], ObservedData2012['water_level_C40_Elev'],'g')#, label='Observed (2014 C37)')
#plt.plot(ObservedData2012['time_C39_mask'], ObservedData2012['water_level_C39_mask'],'r')#, label='Observed (2014 C37)')
plt.plot(T1_C, D1_C,'k-',markersize='4',label='Simulated')

plt.axhline(y=Z_c,linestyle='--',color='g')
#plt.legend()
plt.text(882, 4.6, 'Center, 2012')
plt.xlim(880, 1000)
plt.ylim(4.5, 5.1)

# ------------ 2013 --------------------------------------
ax2 = plt.subplot(323)
plt.fill_between(ObservedData2013['time_C37'], ObservedData2013['water_level_C37']-0.03, 
                 ObservedData2013['water_level_C37']+ 0.03, color='c')
plt.plot(ObservedData2013['time_C37'], ObservedData2013['water_level_C37'],'r')#, label='Trough, 2012')
plt.plot(T1_T, D1_T,'k*',markersize='4')#,label='Simulated')
plt.axhline(y=Z_t,linestyle='--',color='g')
plt.text(1238, 4.6, 'Trough, 2013')
#plt.xlabel('Day')
plt.ylabel('Water level')
plt.xlim(1236, 1362)
plt.ylim(4.5, 5.1)

#plt.legend()

ax1 = plt.subplot(324)

plt.fill_between(ObservedData2013['time_C39'], ObservedData2013['water_level_Center']-0.03, 
                 ObservedData2013['water_level_Center']+ 0.03, color='c')
plt.plot(ObservedData2013['time_C39'], ObservedData2013['water_level_Center'],'r')
plt.plot(T1_C, D1_C,'k*',markersize='4')

plt.text(1238, 4.6, 'Center, 2013')
plt.axhline(y=Z_c,linestyle='--',color='g')
#plt.legend()
plt.xlim(1236, 1362)
plt.ylim(4.5, 5.1)


#---------------- 2014 --------------------
# --------------------------------------------------
ax2 = plt.subplot(325)
plt.fill_between(ObservedData2014['time_C37'], ObservedData2014['water_level_C37']-0.03, 
                 ObservedData2014['water_level_C37']+ 0.03, color='c')
plt.plot(ObservedData2014['time_C37'], ObservedData2014['water_level_C37'],'r')#, label='Observed (trough) 2012')
plt.plot(T1_T, D1_T,'k*',markersize='4')#,label='Simulated')
plt.axhline(y=Z_t,linestyle='--',color='g')
plt.text(1622, 4.6, 'Trough, 2014')
plt.xlabel('Day')
plt.ylabel('Water level')
plt.xlim(1620, 1710)
plt.ylim(4.5, 5.1)
#plt.legend()

ax1 = plt.subplot(326)

plt.fill_between(ObservedData2014['time_C39'], ObservedData2014['water_level_Center']-0.03, 
                 ObservedData2014['water_level_Center']+ 0.03, color='c')
plt.plot(ObservedData2014['time_C39'], ObservedData2014['water_level_Center'],'r',label='Observed')
#plt.plot(ObservedData2014['time_C39'], ObservedData2014['water_level_C39'],'r')#, label='Observed (2014 C37)')
plt.plot(T1_C, D1_C,'k*',markersize='4',label='Simulated')
plt.text(1622, 4.6, 'Center, 2014')
plt.axhline(y=Z_c,linestyle='--',color='g')
plt.legend()
plt.xlim(1620, 1710)
plt.ylim(4.5, 5.1)
"""
ax2 = ax1.twinx()
rain = [d*1.e7 for d in metdata['surface-precipitation_rain']]

ax2.plot(Data40['time'],rain,'y')
ax2.set_ylabel('Precipitation Rain [m /s] x 1.0e-7',color='y')
ax2.tick_params('y', colors='y')
ax2.set_ylim([-10.0, 2])
"""
plt.legend(loc='upper right', fontsize=10, ncol=2, bbox_to_anchor=(0.7,- 0.32, .1, .1))
plt.tight_layout()
plt.savefig(outfile+'comparison-%s.pdf'%simulation, bbox_inches='tight', dpi=100)
plt.show()




In [57]:
935-730, 958-730

(205, 228)